Simulator: HST Up
=================

The majority of simulator scripts are based on Euclid resolution imaging.

This is chosen because it is high enough resolution to resolve the lens and lensed source galaxy's detailed structure
when demonstrating **PyAutoLens** lens modeling features, but low enough resolution for the computation run-time to be
relatively fast.

The instrument simulator scripts in this folder simulate datasets assuming different instruments, which can be used
to gauge the impact of different instrument properties on lens modeling and run-time.

This script simulates imaging representative of Hubble Space Telescope imaging which has been MultiDrizzled to a
higher resolution so that the PSF is better sampled.

__Model__

This script simulates `Imaging` of a 'galaxy-scale' strong lens where:

 - The resolution, PSF and S/N are representative of Hubble Space Telescope `Imaging` that has been multidrizzled to
   higher resolution.

__Start Here Notebook__

If any code in this script is unclear, refer to the `simulators/start_here.ipynb` notebook.

In [ ]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autolens as al
import autolens.plot as aplt

__Dataset Paths__

The path where the dataset will be output.

In [ ]:
dataset_type = "instruments"
dataset_instrument = "hst_up"
dataset_path = path.join("dataset", "imaging", dataset_type, dataset_instrument)

__Simulate__

Simulate the image using a `Grid2D` with the `OverSamplingIterate` object.

In [ ]:
grid = al.Grid2D.uniform(
    shape_native=(260, 260),
    pixel_scales=0.03,
    over_sampling=al.OverSamplingIterate(
        fractional_accuracy=0.9999, sub_steps=[2, 4, 8, 16]
    ),
)

Simulate a simple Gaussian PSF for the image.

In [ ]:
psf = al.Kernel2D.from_gaussian(
    shape_native=(21, 21), sigma=0.05, pixel_scales=grid.pixel_scales, normalize=True
)

To simulate the `Imaging` dataset we first create a simulator, which defines the exposure time, background sky,
noise levels and psf of the dataset that is simulated.

In [ ]:
simulator = al.SimulatorImaging(
    exposure_time=2000.0, psf=psf, background_sky_level=1.0, add_poisson_noise=True
)

__Ray Tracing__

Setup the lens galaxy's mass (SIE+Shear) and source galaxy light (elliptical Sersic) for this simulated lens.

In [ ]:
lens_galaxy = al.Galaxy(
    redshift=0.5,
    mass=al.mp.Isothermal(
        centre=(0.0, 0.0),
        einstein_radius=1.6,
        ell_comps=al.convert.ell_comps_from(axis_ratio=0.8, angle=45.0),
    ),
)

source_galaxy = al.Galaxy(
    redshift=1.0,
    bulge=al.lp.SersicCore(
        centre=(0.1, 0.1),
        ell_comps=al.convert.ell_comps_from(axis_ratio=0.8, angle=60.0),
        intensity=0.3,
        effective_radius=1.0,
        sersic_index=2.5,
    ),
)

Use these galaxies to setup a tracer, which will generate the image for the simulated `Imaging` dataset.

In [ ]:
tracer = al.Tracer(galaxies=[lens_galaxy, source_galaxy])

Lets look at the tracer`s image, this is the image we'll be simulating.

In [ ]:
tracer_plotter = aplt.TracerPlotter(tracer=tracer, grid=grid)
tracer_plotter.figures_2d(image=True)

Pass the simulator a tracer, which creates the image which is simulated as an imaging dataset.

In [ ]:
dataset = simulator.via_tracer_from(tracer=tracer, grid=grid)

__Output__

Output the simulated dataset to the dataset path as .fits files.

In [ ]:
dataset.output_to_fits(
    data_path=path.join(dataset_path, "data.fits"),
    psf_path=path.join(dataset_path, "psf.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    overwrite=True,
)

Plot the simulated `Imaging` dataset before outputting it to fits.

In [ ]:
dataset_plotter = aplt.ImagingPlotter(dataset=dataset)
dataset_plotter.subplot_dataset()

__Visualize__

Output a subplot of the simulated dataset, the image and the tracer's quantities to the dataset path as .png files.

For a faster run time, the tracer visualization uses the binned grid instead of the iterative grid.

In [ ]:
mat_plot = aplt.MatPlot2D(
    title=aplt.Title(label="Hubble Space Telescope Upscaled Image"),
    output=aplt.Output(path=dataset_path, format="png"),
)

dataset_plotter = aplt.ImagingPlotter(dataset=dataset, mat_plot_2d=mat_plot)
dataset_plotter.subplot_dataset()
dataset_plotter.figures_2d(data=True)

tracer_plotter = aplt.TracerPlotter(tracer=tracer, grid=grid, mat_plot_2d=mat_plot)
tracer_plotter.subplot_tracer()
tracer_plotter.subplot_galaxies_images()

__Tracer json__

Save the `Tracer` in the dataset folder as a .json file, ensuring the true light profiles, mass profiles and galaxies
are safely stored and available to check how the dataset was simulated in the future. 

This can be loaded via the method `tracer = al.from_json()`.

In [ ]:
al.output_to_json(
    obj=tracer,
    file_path=path.join(dataset_path, "tracer.json"),
)

The dataset can be viewed in the folder `autolens_workspace/imaging/instruments/hst_up`.